## Step 1: Be able to read in the temperature, pressure, wind speed, wind direction, and time into a pandas dataframe. 


In [2]:
import numpy as np
import pandas as pd
import time
import h5pyd
import geopandas as gpd
from dw_tap.power_output import estimate_power_output
from dw_tap.lom import run_lom

In [3]:
#Using 79m wind speed and direction (hub height is 80m for turbines in this area)
atmospheric_df = pd.read_excel("../data/lidar_marion_OH.xlsx", header=3, usecols="A,L,M,AY,AZ")
atmospheric_df.rename(columns={'Date (UTC)':'datetime', 'Air Temp. Cel.':'temp', 'Pressure (mbar)':'pres', 'Wind Direction (deg) at 79m':'wd', 'Horizontal Wind Speed (m/s) at 79m':'ws'}, inplace=True)
atmospheric_df['temp'] = atmospheric_df['temp'].apply(lambda x : x + 273.15) #convert to K from C
atmospheric_df['pres'] = atmospheric_df['pres'].apply(lambda x : x * 100) #convert to Pascals from mbar
print(atmospheric_df)

                 datetime    temp     pres       wd     ws
0     2017-08-15 14:50:00  304.36  97990.0  272.334  2.110
1     2017-08-15 15:00:00  304.84  97990.0  289.350  2.485
2     2017-08-15 15:10:00  305.71  98000.0  291.621  3.112
3     2017-08-15 15:20:00  306.42  98000.0  298.223  3.143
4     2017-08-15 15:30:00  307.01  98010.0  278.997  4.026
...                   ...     ...      ...      ...    ...
52647 2018-08-16 14:40:00  295.08  98420.0  178.480  8.749
52648 2018-08-16 14:50:00  295.03  98430.0  179.000  8.429
52649 2018-08-16 15:00:00  295.17  98430.0  178.750  8.544
52650 2018-08-16 15:10:00  295.24  98440.0  178.223  9.819
52651 2018-08-16 15:20:00  295.25  98440.0  178.199  9.053

[52652 rows x 5 columns]


In [4]:
#W1 coordinates
z_turbine = 80
lat, lon = 40.591555, -83.182092
obstacle_file = "../sites/simple_marion_obstacles.geojson"
obstacles_df = gpd.read_file(obstacle_file)
obstacles_df = obstacles_df[["height", "geometry"]]
x1_turbine, y1_turbine = lat, lon
xy_turbine = [np.array([x1_turbine, y1_turbine])]

In [5]:
predictions_df = \
    run_lom(atmospheric_df, obstacles_df, xy_turbine, z_turbine).join(atmospheric_df["wd"])

2023-01-10 10:25:07.184578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


LOM time : 3.27  min


In [6]:
print(predictions_df)

                timestamp  ws-adjusted     ws       wd
0     2017-08-15 14:50:00        2.110  2.110  272.334
1     2017-08-15 15:00:00        2.485  2.485  289.350
2     2017-08-15 15:10:00        3.112  3.112  291.621
3     2017-08-15 15:20:00        3.143  3.143  298.223
4     2017-08-15 15:30:00        4.026  4.026  278.997
...                   ...          ...    ...      ...
52647 2018-08-16 14:40:00        8.749  8.749  178.480
52648 2018-08-16 14:50:00        8.429  8.429  179.000
52649 2018-08-16 15:00:00        8.544  8.544  178.750
52650 2018-08-16 15:10:00        9.819  9.819  178.223
52651 2018-08-16 15:20:00        9.053  9.053  178.199

[52652 rows x 4 columns]


In [7]:
kw, above_curve, below_curve, above_curve_list, below_curve_list = \
    estimate_power_output(predictions_df, atmospheric_df["temp"], atmospheric_df["pres"])

In [8]:
print(kw)

0        0.003812
1        0.032116
2        0.106445
3        0.113481
4        0.364442
           ...   
52647    4.453243
52648    3.999863
52649    4.157101
52650    6.222465
52651    4.908551
Length: 52652, dtype: float64


## Step 2: Read in the actual generated kw production from the same time period.

One issue is that the lidar data spans from mid-august of 2017 to mid-august of 2018. It looks like we only have wind production data from 2018 onwards. 

In [9]:
power_output_df = pd.read_excel("../data/turbine.oneenergy.marion.w1.xlsx", header=1, usecols="B")

In [10]:
print(power_output_df)

                     Time
0     2018-01-01 00:00:00
1     2018-01-01 00:10:00
2     2018-01-01 00:20:00
3     2018-01-01 00:30:00
4     2018-01-01 00:40:00
...                   ...
32152 2018-08-18 14:40:00
32153 2018-08-18 14:50:00
32154 2018-08-18 15:00:00
32155 2018-08-18 15:10:00
32156 2018-08-18 15:20:00

[32157 rows x 1 columns]
